In [ ]:
import json

input_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/sft_data/sft_all.jsonl"         # 原始文件路径
output_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/sft_data/sft_steps_3to7.jsonl"  # 输出文件路径

# 统计变量
text_total = 0
text_removed = 0
visual_total = 0
visual_removed = 0

filtered_data = []

with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line.strip())
        images = data.get("images", [])

        # 判断是 text demo 还是 visual demo
        if isinstance(images, list) and len(images) == 1:
            # ---- TEXT DEMO ----
            text_total += 1
            messages = data.get("messages", [])
            user_content = ""
            for msg in messages:
                if msg.get("role") == "user":
                    user_content = msg.get("content", "")
                    break

            # 删除条件：没有 Step 3 且出现 Step 9
            if "Step 9" in user_content and "Step 3" not in user_content:
                text_removed += 1
                continue
            else:
                filtered_data.append(data)

        elif isinstance(images, list) and len(images) >= 2:
            # ---- VISUAL DEMO ----
            visual_total += 1
            if len(images) < 5 or len(images) > 9:
                visual_removed += 1
                continue
            else:
                filtered_data.append(data)

        else:
            # 无法分类的异常数据
            continue

# 写出新文件
with open(output_path, "w", encoding="utf-8") as f_out:
    for d in filtered_data:
        f_out.write(json.dumps(d, ensure_ascii=False) + "\n")

# 打印统计
print("=== 筛选统计报告 ===")
print(f"Text demos:   总数 {text_total} | 删除 {text_removed} | 保留 {text_total - text_removed}")
print(f"Visual demos: 总数 {visual_total} | 删除 {visual_removed} | 保留 {visual_total - visual_removed}")
print(f"最终保留数据共 {len(filtered_data)} 条，已保存到 {output_path}")
